<a href="https://colab.research.google.com/github/Nishimura35/weatherwise-marina-nishimura/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai

import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

# Setup and Configuation
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response
import re

# Add any other setup code here


# Weather Data Functions

def get_user_location():
  """ユーザーから場所の入力を受け取る"""
  pass

def generate_weather_graph(weather_data):
  """matplotlibを使って天気のグラフを作成"""
  pass

def respond_to_user(graph_image, summary_text):
  """ユーザーに結果を返す（グラフ+テキスト）"""
  pass

# Define get_weather_data() function here
def get_weather_data(location: str, forecase_days: int = 3) -> dict:
  if forecast_days < 1 or forecast_days > 5:
    return {"error": "forecast_days must be between 1 and 5"}

  # Convert forecast_days to view_options (0 = 1 day, 1 = 2 days, ..., 4 = 5 days)
  view_option = str(forecast_days - 1)

  # Fetch raw text weather data
  raw_text = fetch_my_weather.get_weather(
    location=location,
    view_option=view_option,
    lang="en"
  )

  if isinstance(raw_text, str) and raw_text.startwith("Error:"):
    return {"error": raw_text}

  # Regex to extract high/low temps and precipitation
  temp_rain_pattern = r'([+\-]?\d+°C)\s*/\s*([+\-]?\d+°C).*?rain:\s*([\d.]+)mm'
  matches = re.findall(temp_rain_pattern, raw_text)

  # Regex to extract weather conditions (e.g., Sunny, Cloudy, Rain)
  condition_pattern = r'\n\s*[☀️🌤️🌧️🌩️❄️☁️⛅️]*\s*([A-Z][a-z]+)'
  conditions = re.findall(condition_pattern, raw_text)

  forecast = []
  for i in range(min(forecast_days, len(matches))):
    high, low, rain = matches[i]
    condition = conditions[i] if i < len(conditions) else "Unknown"
    forecast.append({
      "day": i + 1,
      "condition": condition,
      "high_temp": high,
      "low_temp": low,
      "precipitation_mm": float(rain)
      })

    return {
      "location": location,
      "forecast_days": len(forecast),
      "forecast": forecast
    }


  # Connection error
  except ConnectionError:
    print("Network error. Please check your connection and try again.")
    return None

  # Value error
  except ValueError as ve:
    print(f"Invalid input value: {ve}")
    return None

  # Unexpected error
  except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")
    return None

# Visualisation Functions
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

# Natural Language Processing
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass


def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

# Main function
def main():
  print("Hi I'm Weatherwise App!")
  print("I can help you check the weather forecast.")

  while True:
    try:
      location = input("Which city are you interested in? (type 'exit' to quit)\n")
      if location.lower() == 'exit':
        print("Goodbye! Have a nice day!")
        break

      if not location:
        print("Please enter a city name.")
        continue

      weather_data = get_weather_data(location)
      if not weather_data or 'error' in weather_data:
        print(f"Sorry, I couldn't find weather data for '{location}'. Please try another city.")
        continue

      create_temperature_visualisation(weather_data)
      create_precipitation_visualisation(weather_data)

    except ConnectionError:
      print("It looks like there's a problem with your internet connection. Please check and try again.")
    except Exception as e:
      print("Sorry, I couldn't fetch the weather data. Please try again.\n")

  except Exception as e:
    print(f"Oops! Something went wrong: {e}")
    print("Please try again or contact support if the issue persists.")

# User Interface


# Main Application Logic


if __name__ == '__main__':
    main()



# Testing and Examples
# Include sample input/output for each function